In [6]:
#commencer ici avec jupyter
from pyspark import SparkContext
from pyspark.sql import SparkSession

spark = SparkSession\
        .builder\
        .master('local[*]')\
        .getOrCreate()
sc = spark.sparkContext

ratings=sc.textFile('movielens/ratings.dat')
ratings.take(5)

['1::1193::5::978300760',
 '1::661::3::978302109',
 '1::914::3::978301968',
 '1::3408::4::978300275',
 '1::2355::5::978824291']

In [7]:
def parseRatings(row):
    splitted = list(row.split('::'))
    return (int(splitted[0]),int(splitted[1]),int(splitted[2]),splitted[3])

In [8]:
ratings=ratings.map(parseRatings)
ratings.take(5)

[(1, 1193, 5, '978300760'),
 (1, 661, 3, '978302109'),
 (1, 914, 3, '978301968'),
 (1, 3408, 4, '978300275'),
 (1, 2355, 5, '978824291')]

In [10]:
ratings.filter(lambda x:x[2]==1).count()

56174

In [12]:
ratings.map(lambda x:x[1]).distinct().count()

3706

In [20]:
ratings.map(lambda x:(x[0],1)).reduceByKey(lambda x,y:x+y).max(lambda x:x[1])

(4169, 2314)

In [30]:
movies=sc.textFile('movielens/movies.dat')
movies.take(5)

["1::Toy Story (1995)::Animation|Children's|Comedy",
 "2::Jumanji (1995)::Adventure|Children's|Fantasy",
 '3::Grumpier Old Men (1995)::Comedy|Romance',
 '4::Waiting to Exhale (1995)::Comedy|Drama',
 '5::Father of the Bride Part II (1995)::Comedy']

In [37]:
users=sc.textFile('movielens/users.dat')
users.take(5)

['1::F::1::10::48067',
 '2::M::56::16::70072',
 '3::M::25::15::55117',
 '4::M::45::7::02460',
 '5::M::25::20::55455']

In [25]:
def parseMovies(row):
    splitted = list(row.split('::'))
    return (int(splitted[0]),splitted[1],splitted[2])

In [31]:
movies=movies.map(parseMovies)
movies.take(5)

[(1, 'Toy Story (1995)', "Animation|Children's|Comedy"),
 (2, 'Jumanji (1995)', "Adventure|Children's|Fantasy"),
 (3, 'Grumpier Old Men (1995)', 'Comedy|Romance'),
 (4, 'Waiting to Exhale (1995)', 'Comedy|Drama'),
 (5, 'Father of the Bride Part II (1995)', 'Comedy')]

In [35]:
def parseUsers(row):
    splitted = list(row.split('::'))
    return (int(splitted[0]),splitted[1],int(splitted[2]), splitted[3],splitted[4])

In [38]:
users=users.map(parseUsers)
users.take(5)

[(1, 'F', 1, '10', '48067'),
 (2, 'M', 56, '16', '70072'),
 (3, 'M', 25, '15', '55117'),
 (4, 'M', 45, '7', '02460'),
 (5, 'M', 25, '20', '55455')]

In [41]:
users.sortBy(lambda x:x,0).distinct().take(5)

[(6039, 'F', 45, '0', '01060'),
 (6037, 'F', 45, '1', '76006'),
 (6036, 'F', 25, '15', '32603'),
 (6033, 'M', 50, '13', '78232'),
 (6028, 'M', 18, '4', '94133')]

In [50]:
moviesGenres=movies.flatMap(lambda x:x[2].split('|')).map(lambda x:(x,1)).reduceByKey(lambda x,y:x+y)
moviesGenres.take(5)

[("Children's", 251),
 ('Fantasy', 68),
 ('Romance', 471),
 ('Drama', 1603),
 ('Action', 503)]